In [1]:
import random
import simpy

In [2]:
RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME = 5      # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~7 minutes
SIM_TIME = 20     # Simulation time in minutes

In [3]:
class Carwash:
    """A carwash has a limited number of machines to clean cars in parallel.
    Cars have to request one of the machines(``NUM_MACHINES``). When they get one, they can start
    the washing process and wait for it to finish (which takes ``washtime`` minutes.)
    
    """
    def __init__(self, env, num_machines, washtime=WASHTIME):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime
    
    def wash(self, car):
        yield self.env.timeout(self.washtime)
        print(f"Carwash removed {random.randint(50, 99)}% of {car}'s dirt.")
        
def car(env, name, cw):
    """The car process (each car has a ``name``) arrives at the carwash
    (``cw``) and requests a cleaning machinne
    
    It then starts the washing process, waits for it to finish and 
    leaves to never come back...
    """
    print(f"{name} arrives at the carwash at {float(env.now)}")
    with cw.machine.request() as request:
        yield request
        
        print(f'{name} enters the carwash at {float(env.now)}')
        yield env.process(cw.wash(name))
        
        print(f'{name} leaves the carwash at {float(env.now)}')
        
def setup(env, num_machines, washtime, t_inter):
    """Create a carwash, a number of initial cars and keep creating cars approx
    every ``t_integer`` minutes."""
    # Create the carwash
    carwash = Carwash(env, num_machines, washtime)
    
    # Create 4 initial cars
    for i in range(4):
        env.process(car(env, f'Car {i}', carwash))
        
    # Create more cars while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, f'Car {i}', carwash))
        
# Setup and start the simulation
print('Carwash')
random.seed(RANDOM_SEED)

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

# Execute
env.run(until=SIM_TIME)

Carwash
Car 0 arrives at the carwash at 0.0
Car 1 arrives at the carwash at 0.0
Car 2 arrives at the carwash at 0.0
Car 3 arrives at the carwash at 0.0
Car 0 enters the carwash at 0.0
Car 1 enters the carwash at 0.0
Car 4 arrives at the carwash at 5.0
Carwash removed 97% of Car 0's dirt.
Carwash removed 67% of Car 1's dirt.
Car 0 leaves the carwash at 5.0
Car 1 leaves the carwash at 5.0
Car 2 enters the carwash at 5.0
Car 3 enters the carwash at 5.0
Car 5 arrives at the carwash at 10.0
Carwash removed 64% of Car 2's dirt.
Carwash removed 58% of Car 3's dirt.
Car 2 leaves the carwash at 10.0
Car 3 leaves the carwash at 10.0
Car 4 enters the carwash at 10.0
Car 5 enters the carwash at 10.0
Carwash removed 97% of Car 4's dirt.
Carwash removed 56% of Car 5's dirt.
Car 4 leaves the carwash at 15.0
Car 5 leaves the carwash at 15.0
Car 6 arrives at the carwash at 16.0
Car 6 enters the carwash at 16.0
